In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.interpolate import make_interp_spline
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import gaussian_kde

In [ ]:
df = pd.read_csv('../result/output_scenario.csv')
df['pop'] = np.where(df['pop'] > 500, 500, df['pop'])
df['total_cost_0_ave'] = df['total_cost_0'] / df['pop']
df['total_cost_2020_ave'] = df['total_cost_2020'] / df['pop']
df['total_cost_2050_ave'] = df['total_cost_2050'] / df['pop']
df['diff'] = (df['total_cost_2020_ave'] - df['total_cost_0_ave']) / df['total_cost_0_ave']
df

In [ ]:
print(df['total_cost_0_ave'].describe())

In [ ]:
print(df['diff'].describe())

In [ ]:
def visualize_cost(data, vmax, column):
    plt.rcParams['font.family'] = 'Times New Roman'
    # 导入必要的模块
    from matplotlib.offsetbox import OffsetImage, AnnotationBbox
    import numpy as np
    from scipy.ndimage import gaussian_filter
    
    # 创建Point几何图形
    data['geometry'] = data.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    geo_df = gpd.GeoDataFrame(data, geometry='geometry')
    geo_df.set_crs(epsg=4326, inplace=True)
    
    # 创建带有cartopy投影的图形
    fig = plt.figure(figsize=(14, 6), dpi=500)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

    # 设置背景和地图特征
    ax.set_facecolor('#CCCCCC')  
    ax.add_feature(cfeature.LAND, color="#CECECE", alpha=0.4)
    ax.add_feature(cfeature.OCEAN, color="#FFFFFF", alpha=0.5)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.3)
    
    # 使用更简单的方法创建密度图
    resolution = 1.0  # 1度分辨率
    lons = np.arange(-180, 181, resolution)
    lats = np.arange(-90, 91, resolution)
    
    # 创建网格点
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    density_grid = np.zeros_like(lon_grid)
    
    # 提取所有点的经纬度和值
    all_lons = geo_df.geometry.x.values
    all_lats = geo_df.geometry.y.values
    all_values = geo_df[column].values
    
    # 使用更适合地理数据的影响半径
    bandwidth = 2.0
    
    # 计算每个网格点的密度值
    for i, row in enumerate(all_lons):
        lon, lat = all_lons[i], all_lats[i]
        value = all_values[i]
        
        # 计算此数据点对网格的影响
        for x in range(len(lats)):
            for y in range(len(lons)):
                grid_lon = lons[y]
                grid_lat = lats[x]
                
                # 计算距离
                dist = np.sqrt((grid_lon - lon)**2 + (grid_lat - lat)**2)
                
                # 只在一定半径内影响网格点
                if dist <= 3 * bandwidth:
                    # 使用高斯核计算权重
                    weight = np.exp(-0.5 * (dist / bandwidth)**2) * value
                    density_grid[x, y] += weight
    
    # 对密度图应用轻微的高斯平滑
    density_grid = gaussian_filter(density_grid, sigma=1.0)
    
    # 设置阈值，只在密度足够高的区域绘制斜线
    threshold = np.max(density_grid) * 0.2
    mask = density_grid < threshold
    
    # 设置上限值，避免极端值影响色彩分布
    # 使用百分位数而不是最大值来设置上限，避免离群值的影响
    upper_limit = np.percentile(density_grid[~mask], 99)  # 使用99%分位数作为上限
    
    # 裁剪过高的值
    density_grid_capped = np.clip(density_grid, 0, upper_limit)
    
    # 创建掩码数组
    density_grid_masked = np.ma.array(density_grid_capped, mask=mask)
    
    # 热力图颜色 - 使用更多颜色变化
    heatmap_colors = ['#012f4830', '#669aba40', '#fbf0d950', '#be142060', '#7a010170']
    heatmap_cmap = mpl.colors.LinearSegmentedColormap.from_list('heatmap_palette', heatmap_colors)
    
    # 创建自定义规范化对象，确保色彩分布均匀
    heatmap_norm = mpl.colors.Normalize(vmin=threshold, vmax=upper_limit)
    
    # 绘制密度图 - 只在有数据的区域绘制斜线
    contour = ax.contourf(
        lon_grid, lat_grid, density_grid_masked,
        transform=ccrs.PlateCarree(),
        cmap=heatmap_cmap,
        norm=heatmap_norm,  # 使用自定义规范化
        levels=8,
        alpha=0.7,
        zorder=2,
        # hatches=['/////', '/////', '/////', '/////', '/////', '/////', '/////', '/////'],
        extend='neither'
    )
    
    # 设置颜色映射
    sci_colors = ['#012f48', '#669aba', '#fbf0d9', '#be1420', '#7a0101']  # From blue to red
    custom_cmap = mpl.colors.LinearSegmentedColormap.from_list('sci_palette', sci_colors)
    norm = mpl.colors.Normalize(vmin=300, vmax=vmax)
    
    # 计算点大小范围
    min_size = 20
    max_size = 200
    
    # 对数据按值排序，保证小值在下层，大值在上层
    geo_df = geo_df.sort_values(by=column)
    
    # 创建一个假散点用于颜色条
    fake_scatter = ax.scatter(
        [-1000], [-1000],  # 不可见位置
        c=[0],
        cmap=custom_cmap,
        vmin=300,
        vmax=vmax,
        s=1
    )
    
    # 为每个点创建并添加标记
    for idx, row in geo_df.iterrows():
        # 获取经纬度和值
        lon, lat = row.geometry.x, row.geometry.y
        value = row[column]
        
        # 计算点大小
        size = min_size + ((value / vmax) * (max_size - min_size))
        
        # 计算颜色
        color = custom_cmap(norm(value))
        
        # 创建临时图形来生成点图像
        temp_fig = plt.figure(figsize=(1, 1), frameon=False, dpi= 300)
        temp_fig.patch.set_alpha(0)  # 透明背景
        
        temp_ax = temp_fig.add_subplot(111)
        temp_ax.set_aspect('equal')
        temp_ax.patch.set_alpha(0)
        
        # 先绘制一个稍大的黑色圆作为边框
        outer_circle = plt.Circle(
            (0.5, 0.5),
            0.18,         # 稍大的半径
            color='black',  # 黑色
            alpha=1
        )
        temp_ax.add_patch(outer_circle)

        # 再绘制一个内圆作为主要颜色区域
        inner_circle = plt.Circle(
            (0.5, 0.5),
            0.15,         # 稍小的半径
            color=color,  # 数据颜色
            alpha=1
        )
        temp_ax.add_patch(inner_circle)
        
        # 设置坐标轴范围
        temp_ax.set_xlim(0, 1)
        temp_ax.set_ylim(0, 1)
        temp_ax.axis('off')  # 隐藏坐标轴
        
        # 渲染并获取图像
        temp_fig.tight_layout(pad=0)
        temp_fig.canvas.draw()
        point_img = np.array(temp_fig.canvas.renderer.buffer_rgba())
        plt.close(temp_fig)
        
        # 将地理坐标转换为投影坐标
        x, y = ax.projection.transform_point(lon, lat, src_crs=ccrs.PlateCarree())
        
        # 计算缩放因子 - 调整点大小
        zoom_factor = np.sqrt(size) / 250
        
        # 创建OffsetImage
        imagebox = OffsetImage(point_img, zoom=zoom_factor)
        imagebox.image.axes = ax
        
        # 创建并添加AnnotationBbox
        ab = AnnotationBbox(
            imagebox,
            (x, y),
            frameon=False,
            pad=0,
            zorder=10  # 确保点在最上层
        )
        ax.add_artist(ab)
    
    # 添加颜色条
    cbar = fig.colorbar(
        fake_scatter,
        ax=ax,
        orientation='horizontal',
        shrink=0.6,  # 控制颜色条长度
        pad=0.03,    # 调整颜色条和图形之间的间距
        aspect=50    # 控制颜色条的宽度（值越小越宽）
    )
    cbar.set_label(f'Per Capita Investment($)', fontsize=12)
    
    # 设置全球边界
    ax.set_global()
     
    # 移除坐标轴刻度和边框
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.axis('off')  # 完全关闭坐标轴
    
    # 减少边距
    plt.tight_layout(pad=0)
    
    plt.show()

In [ ]:
vmax = 1e3
visualize_cost(df, vmax, 'total_cost_0_ave')
# visualize_cost(df, vmax, 'total_cost_2020_ave')
# visualize_cost(df, vmax, 'total_cost_2050_ave')

In [ ]:
def visualize_diff(data, vmax, column):
    plt.rcParams['font.family'] = 'Times New Roman'
    # 导入必要的模块
    from matplotlib.offsetbox import OffsetImage, AnnotationBbox
    import numpy as np
    from scipy.ndimage import gaussian_filter
    
    # 创建Point几何图形
    data['geometry'] = data.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    geo_df = gpd.GeoDataFrame(data, geometry='geometry')
    geo_df.set_crs(epsg=4326, inplace=True)
    
    # 创建带有cartopy投影的图形
    fig = plt.figure(figsize=(14, 6), dpi=500)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

    # 设置背景和地图特征
    ax.set_facecolor('#CCCCCC')  
    ax.add_feature(cfeature.LAND, color="#CECECE", alpha=0.4)
    ax.add_feature(cfeature.OCEAN, color="#FFFFFF", alpha=0.5)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, alpha=0.3)
    
    # 使用更简单的方法创建密度图
    resolution = 1.0  # 1度分辨率
    lons = np.arange(-180, 181, resolution)
    lats = np.arange(-90, 91, resolution)
    
    # 创建网格点
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    density_grid = np.zeros_like(lon_grid)
    
    # 提取所有点的经纬度和值
    all_lons = geo_df.geometry.x.values
    all_lats = geo_df.geometry.y.values
    all_values = geo_df[column].values
    
    # 使用更适合地理数据的影响半径
    bandwidth = 3.0
    
    # 计算每个网格点的密度值
    for i, row in enumerate(all_lons):
        lon, lat = all_lons[i], all_lats[i]
        value = all_values[i]
        
        # 计算此数据点对网格的影响
        for x in range(len(lats)):
            for y in range(len(lons)):
                grid_lon = lons[y]
                grid_lat = lats[x]
                
                # 计算距离
                dist = np.sqrt((grid_lon - lon)**2 + (grid_lat - lat)**2)
                
                # 只在一定半径内影响网格点
                if dist <= 3 * bandwidth:
                    # 使用高斯核计算权重
                    weight = np.exp(-0.5 * (dist / bandwidth)**2) * value
                    density_grid[x, y] += weight
    
    # 对密度图应用轻微的高斯平滑
    density_grid = gaussian_filter(density_grid, sigma=1.0)
    
    # 设置阈值，只在密度足够高的区域绘制斜线
    threshold = np.max(density_grid) * 0.15
    mask = density_grid < threshold
    
    # 设置上限值，避免极端值影响色彩分布
    # 使用百分位数而不是最大值来设置上限，避免离群值的影响
    upper_limit = np.percentile(density_grid[~mask], 99)  # 使用99%分位数作为上限
    
    # 裁剪过高的值
    density_grid_capped = np.clip(density_grid, 0, upper_limit)
    
    # 创建掩码数组
    density_grid_masked = np.ma.array(density_grid_capped, mask=mask)
    
    # 热力图颜色 - 使用更多颜色变化
    heatmap_colors = ['#012f4830', '#669aba40', '#fbf0d950', '#be142060', '#7a010170']
    heatmap_cmap = mpl.colors.LinearSegmentedColormap.from_list('heatmap_palette', heatmap_colors)
    
    # 创建自定义规范化对象，确保色彩分布均匀
    heatmap_norm = mpl.colors.Normalize(vmin=threshold, vmax=upper_limit)
    
    # 绘制密度图 - 只在有数据的区域绘制斜线
    contour = ax.contourf(
        lon_grid, lat_grid, density_grid_masked,
        transform=ccrs.PlateCarree(),
        cmap=heatmap_cmap,
        norm=heatmap_norm,  # 使用自定义规范化
        levels=8,
        alpha=0.7,
        zorder=2,
        # hatches=['/////', '/////', '/////', '/////', '/////', '/////', '/////', '/////'],
        extend='neither'
    )
    
    # 设置颜色映射
    sci_colors = ['#012f48', '#669aba', '#fbf0d9', '#be1420', '#7a0101']  # From blue to red
    custom_cmap = mpl.colors.LinearSegmentedColormap.from_list('sci_palette', sci_colors)
    norm = mpl.colors.Normalize(vmin=0, vmax=vmax)
    
    # 计算点大小范围
    min_size = 40
    max_size = 200
    
    # 对数据按值排序，保证小值在下层，大值在上层
    geo_df = geo_df.sort_values(by=column)
    
    # 创建一个假散点用于颜色条
    fake_scatter = ax.scatter(
        [-1000], [-1000],  # 不可见位置
        c=[0],
        cmap=custom_cmap,
        vmin=-vmax,
        vmax=vmax,
        s=1
    )
    
    # 为每个点创建并添加标记
    for idx, row in geo_df.iterrows():
        # 获取经纬度和值
        lon, lat = row.geometry.x, row.geometry.y
        value = row[column]
        
        # 计算点大小
        size = min_size + ((value / vmax) * (max_size - min_size))
        
        # 计算颜色
        color = custom_cmap(norm(value))
        
        # 创建临时图形来生成点图像
        temp_fig = plt.figure(figsize=(1, 1), frameon=False, dpi= 300)
        temp_fig.patch.set_alpha(0)  # 透明背景
        
        temp_ax = temp_fig.add_subplot(111)
        temp_ax.set_aspect('equal')
        temp_ax.patch.set_alpha(0)
        
        # 先绘制一个稍大的黑色圆作为边框
        outer_circle = plt.Circle(
            (0.5, 0.5),
            0.18,         # 稍大的半径
            color='black',  # 黑色
            alpha=1
        )
        temp_ax.add_patch(outer_circle)

        # 再绘制一个内圆作为主要颜色区域
        inner_circle = plt.Circle(
            (0.5, 0.5),
            0.15,         # 稍小的半径
            color=color,  # 数据颜色
            alpha=1
        )
        temp_ax.add_patch(inner_circle)
        
        # 设置坐标轴范围
        temp_ax.set_xlim(0, 1)
        temp_ax.set_ylim(0, 1)
        temp_ax.axis('off')  # 隐藏坐标轴
        
        # 渲染并获取图像
        temp_fig.tight_layout(pad=0)
        temp_fig.canvas.draw()
        point_img = np.array(temp_fig.canvas.renderer.buffer_rgba())
        plt.close(temp_fig)
        
        # 将地理坐标转换为投影坐标
        x, y = ax.projection.transform_point(lon, lat, src_crs=ccrs.PlateCarree())
        
        # 计算缩放因子 - 调整点大小
        zoom_factor = np.sqrt(size) / 250
        
        # 创建OffsetImage
        imagebox = OffsetImage(point_img, zoom=zoom_factor)
        imagebox.image.axes = ax
        
        # 创建并添加AnnotationBbox
        ab = AnnotationBbox(
            imagebox,
            (x, y),
            frameon=False,
            pad=0,
            zorder=10  # 确保点在最上层
        )
        ax.add_artist(ab)
    
    # 添加颜色条
    cbar = fig.colorbar(
        fake_scatter,
        ax=ax,
        orientation='horizontal',
        shrink=0.6,  # 控制颜色条长度
        pad=0.03,    # 调整颜色条和图形之间的间距
        aspect=50    # 控制颜色条的宽度（值越小越宽）
    )
    cbar.set_label(f'Diff', fontsize=12)
    
    # 设置全球边界
    ax.set_global()
     
    # 移除坐标轴刻度和边框
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.axis('off')  # 完全关闭坐标轴
    
    # 减少边距
    plt.tight_layout(pad=0)
    
    plt.show()

In [ ]:
vmax = 0.5
visualize_diff(df, vmax, 'diff')

In [ ]:
def create_longitude_profile(data, column, ax=None):
    plt.rcParams['font.family'] = 'Times New Roman'
    """创建经度方向的曲线图"""
    if ax is None:
        fig, ax = plt.subplots(figsize=(14, 1.5),dpi=500)
    
    # 按经度分组计算平均值
    lon_bins = np.arange(-180, 181, 5)  # 5度一组
    lon_centers = (lon_bins[:-1] + lon_bins[1:]) / 2
    lon_values = []
    
    for i in range(len(lon_bins)-1):
        mask = (data['lon'] >= lon_bins[i]) & (data['lon'] < lon_bins[i+1])
        if mask.any():
            lon_values.append(data.loc[mask, column].mean())
        else:
            lon_values.append(0)  # 用0代替NaN
    
    # 平滑曲线
    if len(lon_centers) > 3:
        lon_smooth = np.linspace(min(lon_centers), max(lon_centers), 300)
        spline = make_interp_spline(lon_centers, lon_values, k=3)
        lon_values_smooth = spline(lon_smooth)
        # 确保没有负值
        lon_values_smooth = np.maximum(lon_values_smooth, 0)
    else:
        lon_smooth = lon_centers
        lon_values_smooth = lon_values
    
    # 绘制曲线
    ax.plot(lon_smooth, lon_values_smooth, color='#7a0101', linewidth=1.5)
    ax.fill_between(lon_smooth, 0, lon_values_smooth, alpha=0.3, color="#a37070")
    
    # 设置范围和网格线
    ax.set_xlim(-180, 180)
    ax.set_ylim(0, None)
    
    # 添加网格线
    # ax.axvline(x=0, color='gray', linestyle='-', alpha=0.3, linewidth=0.5)
    # ax.axvline(x=60, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    # ax.axvline(x=120, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    # ax.axvline(x=-60, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    # ax.axvline(x=-120, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    
    # 添加经度标签
    # ax.text(0, -0.05, "0°", transform=ax.transData, ha='center', va='top', fontsize=8)
    # ax.text(60, -0.05, "60°E", transform=ax.transData, ha='center', va='top', fontsize=8)
    # ax.text(120, -0.05, "120°E", transform=ax.transData, ha='center', va='top', fontsize=8)
    # ax.text(-60, -0.05, "60°W", transform=ax.transData, ha='center', va='top', fontsize=8)
    # ax.text(-120, -0.05, "120°W", transform=ax.transData, ha='center', va='top', fontsize=8)
    
    # 美化图表
    ax.set_xticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    # ax.set_ylabel('Avg. Investment', fontsize=9)
    
    return ax

def create_latitude_profile(data, column, ax=None):
    """创建纬度方向的曲线图"""
    if ax is None:
        fig, ax = plt.subplots(figsize=(2, 10),dpi=500)
    
    # 按纬度分组计算平均值
    lat_bins = np.arange(-90, 91, 2)  # 2度一组
    lat_centers = (lat_bins[:-1] + lat_bins[1:]) / 2
    lat_values = []
    
    for i in range(len(lat_bins)-1):
        mask = (data['lat'] >= lat_bins[i]) & (data['lat'] < lat_bins[i+1])
        if mask.any():
            lat_values.append(data.loc[mask, column].mean())
        else:
            lat_values.append(0)  # 用0代替NaN
    
    # 平滑曲线
    if len(lat_centers) > 3:
        lat_smooth = np.linspace(min(lat_centers), max(lat_centers), 300)
        spline = make_interp_spline(lat_centers, lat_values, k=3)
        lat_values_smooth = spline(lat_smooth)
        # 确保没有负值
        lat_values_smooth = np.maximum(lat_values_smooth, 0)
    else:
        lat_smooth = lat_centers
        lat_values_smooth = lat_values
    
    # 绘制曲线
    ax.plot(lat_values_smooth, lat_smooth, color='#7a0101', linewidth=1.5)
    ax.fill_betweenx(lat_smooth, 0, lat_values_smooth, alpha=0.3, color='#a37070')
    
    # 设置范围和网格线
    ax.set_ylim(-90, 90)
    ax.set_xlim(0, None)
    
    # 添加网格线
    # ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3, linewidth=0.5)
    # ax.axhline(y=30, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    # ax.axhline(y=60, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    # ax.axhline(y=-30, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    # ax.axhline(y=-60, color='gray', linestyle='--', alpha=0.3, linewidth=0.5)
    
    # 添加纬度标签
    # ax.text(-0.05, 0, "0°", transform=ax.transData, ha='right', va='center', fontsize=8)
    # ax.text(-0.05, 30, "30°N", transform=ax.transData, ha='right', va='center', fontsize=8)
    # ax.text(-0.05, 60, "60°N", transform=ax.transData, ha='right', va='center', fontsize=8)
    # ax.text(-0.05, -30, "30°S", transform=ax.transData, ha='right', va='center', fontsize=8)
    # ax.text(-0.05, -60, "60°S", transform=ax.transData, ha='right', va='center', fontsize=8)
    
    # 美化图表
    ax.set_yticks([])
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    # ax.set_xlabel('Avg. Investment', fontsize=9)
    
    return ax

In [ ]:
create_longitude_profile(df, 'total_cost_0_ave', ax=None)
create_latitude_profile(df, 'total_cost_0_ave', ax=None)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 设置全局字体和样式，使图形更专业
plt.rcParams.update({
    'font.family': 'Times New Roman',  # 设置字体
    'font.size': 16,                  # 全局字体大小
    'axes.titlesize': 16,             # 坐标轴标题大小
    'axes.labelsize': 16,             # 坐标轴标签大小
    'xtick.labelsize': 16,            # x轴刻度字体大小
    'ytick.labelsize': 16,            # y轴刻度字体大小
    'legend.fontsize': 12,            # 图例字体大小
    'figure.titlesize': 16            # 图形标题字体大小
})

# 读取真实的岛屿成本数据
real_cost_df = pd.read_csv('island_cost_summary_0.csv')

# 使用人均成本数据
cost_columns = ['renewable_cost_per_capita', 'storage_cost_per_capita', 'lng_cost_per_capita', 
               'other_equipment_cost_per_capita', 'discard_cost_per_capita', 'load_shedding_cost_per_capita']
 
# 重命名列以匹配原代码的期望
column_mapping = {
    'renewable_cost_per_capita': "Renewable Cost",
    'storage_cost_per_capita': "Storage Cost", 
    'lng_cost_per_capita': "LNG Cost",
    'other_equipment_cost_per_capita': "Other Equipment Cost",
    'discard_cost_per_capita': "Discard Cost",
    'load_shedding_cost_per_capita': "Load Shedding Cost"
}

# 筛选有足够数据的区域
region_counts = real_cost_df['IPCC_Region_Code'].value_counts()
valid_regions = region_counts[region_counts >= 10].index.tolist()

# 筛选数据
filtered_df = real_cost_df[real_cost_df['IPCC_Region_Code'].isin(valid_regions)].copy()

# 重命名成本列
for old_col, new_col in column_mapping.items():
    if old_col in filtered_df.columns:
        filtered_df[new_col] = filtered_df[old_col]

# 按区域平均成本排序区域
# 先计算每个区域的总人均成本，然后按平均值排序
filtered_df['total_cost_per_capita'] = filtered_df[list(column_mapping.values())].sum(axis=1)
total_cost_per_region = filtered_df.groupby('IPCC_Region_Code')['total_cost_per_capita'].mean()
areas = total_cost_per_region.sort_values(ascending=False).index.tolist()

# 获取成本字段
cost_names = list(column_mapping.values())

# 重命名区域列以匹配原代码
filtered_df['Area'] = filtered_df['IPCC_Region_Code']

# 定义成本类型与颜色的映射字典
cost_color_map = {
    "Renewable Cost": "#4C72B0",
    "Storage Cost": "#55A868",
    "LNG Cost": "#808080",
    "Other Equipment Cost": "#8172B3",
    "Discard Cost": "#d47d49",
    "Load Shedding Cost": "#C44E52"
}

print(f"使用 {len(filtered_df)} 个岛屿的数据")
print(f"包含区域: {areas}")
print(f"成本类型: {cost_names}")

# 遍历每种成本类型并生成单独的图
for cost_name in cost_names:
    # 获取对应的颜色
    current_color = cost_color_map[cost_name]
    
    # 准备当前成本类别的数据
    area_groups = filtered_df.groupby('Area')[cost_name].apply(list)
    # 确保区域顺序正确
    area_groups = area_groups.reindex(areas)

    # 创建图形和坐标轴
    fig, ax = plt.subplots(figsize=(16, 4), dpi=300)

    # 定义x轴上的位置
    positions = np.arange(len(areas)) + 1

    # 为每个区域创建组合雨云图
    for i, area in enumerate(areas):
        if area not in area_groups.index or len(area_groups[area]) == 0:
            continue
            
        # A. 半小提琴图（云部分）
        violin_parts = ax.violinplot(
            area_groups[area], 
            positions=[positions[i]], 
            showmeans=False, 
            showmedians=False, 
            showextrema=False
        )
        
        # 修改小提琴图，只显示右半部分
        for pc in violin_parts['bodies']:
            vertices = pc.get_paths()[0].vertices
            center_x = positions[i]
            vertices[vertices[:, 0] < center_x, 0] = center_x
            pc.set_facecolor(current_color)  # 设置颜色
            pc.set_edgecolor('black')
            pc.set_alpha(0.8)

        # B. 箱线图（雨部分）
        ax.boxplot(
            area_groups[area], 
            positions=[positions[i]], 
            widths=0.18,  # 调整宽度
            patch_artist=True,
            boxprops=dict(facecolor='white', color='black'),
            medianprops=dict(color='#be1420', linewidth=3),
            whiskerprops=dict(color='black', linewidth=1),  # 调整须线粗细
            capprops=dict(color='black', linewidth=1),      # 调整须线端点横线粗细
            flierprops=dict(marker='o', color='black', markersize=4, markerfacecolor='gray'),
            showfliers=True
        )

        # C. 散点图（雨滴部分）
        jittered_x = np.random.normal(positions[i] - 0.15, 0.03, size=len(area_groups[area]))
        ax.scatter(
            jittered_x, 
            area_groups[area], 
            color=current_color,  # 设置颜色
            alpha=0.7, 
            s=20,
            edgecolors='black', 
            linewidths=0.5
        )

    # --- 格式化和样式 ---
    
    # 设置动态标题和标签
    # ax.set_title(f'Distribution of {cost_name} by Area')  # 可选标题
    ax.set_ylabel(f'{cost_name}($/per)')

    # 设置x轴刻度和标签
    ax.set_xticks(positions[:len(areas)])
    ax.set_xticklabels(areas, rotation=30, ha='right')

    # 隐藏顶部和右侧边框
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # 添加轻微的网格线以提高可读性
    ax.grid(axis='y', linestyle='--', alpha=0.3)

    # 调整布局以防止标签重叠
    plt.tight_layout()

    # 显示当前成本类别的图
    plt.show()